<h2 align='center'> MÔN HỌC: KỸ NGHỆ TRI THỨC VÀ HỌC MÁY (7080510) </h2>

---
* Giảng viên: Đặng Văn Nam
* Email: dangvannam@humg.edu.n
---

### CHƯƠNG 5: HỆ THỐNG GỢI Ý (RECOMMENDER SYSTEMS)
---
**NỘI DUNG BÀI HỌC:**
1. Giới thiệu chung
2. Phân loại hệ thống đề xuất
3. Các phương pháp tính toán độ tương đồng
4. Sơ đồ tổng quan và Thách thức
5. Ví dụ minh họa
---
dangvannam@Department of Computer Science@2020

---
### 5. VÍ DỤ HỆ THỐNG GỢI Ý MOVIES
Dự án này sẽ xây dựng một hệ thống đề xuất dựa trên tập dữ liệu Movies.

<img src='Pic/pic1.png'>

Dựa vào dữ liệu của trên 12 182 bộ films, xây dựng hệ thống đề xuất đưa ra danh sách 15 bộ film liên quan. Có hai loại Recommender system được xây dựng trong project này:


*   Simple Recommender
*   Content-Based Recommender

Các file dữ liệu sử dụng bao gồm:

**Data_Movies.csv:** File này chứa thông tin tổng hợp của ~ 12 000 bộ film, mỗi bộ film có 24 thuộc tính khác nhau, một số thuộc tính chính bao gồm:

1. adult: Bộ film dành cho người lớn hay không. Dữ liệu boolean (True - Flase)
2. original_language: Ngôn ngữ ban đầu; dữ liệu categorical
3. genres: Thể loại film
4. original_title: Tiêu đề của film, dữ liệu text
5. overview: Tóm tắt nội dung của film; Dữ liệu text
6. release_date: Ngày phát hành films
7. vote_average: Tỷ lệ vote trung bình [0-10]
8. vote_count: Số lượt vote

## I) Đọc tập dữ liệu Movie
---

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Đọc tập dữ liệu thông tin của các film
path='data\Data_Movies.csv'
data_movies = pd.read_csv(path)

#Hiển thị thông tin tập dữ liệu
data_movies.info()

In [ ]:
#Hiển thị dữ liệu 5 dòng đầu tiên
data_movies.head()

**Lọc dữ liệu thô ban đầu chỉ lấy các cột quan trọng sử dụng để xây dựng hệ thống gợi ý:**
* adult: Bộ film dành cho người lớn hay không. Dữ liệu boolean (True - Flase)
* original_language: Ngôn ngữ ban đầu; dữ liệu categorical
* original_title: Tiêu đề của film, dữ liệu text
* overview: Tóm tắt nội dung của film; Dữ liệu text
* release_date: Ngày phát hành film
* vote_average: Tỷ lệ vote trung bình [0-10]
* vote_count: Số lượt vote

In [ ]:
data = data_movies.loc[:,['adult',
                          'original_title',
                          'overview',
                          'release_date',
                          'vote_average',
                          'vote_count']].copy()
data.info()

In [ ]:
data.head()

In [ ]:
#Đặc trưng thống kê các thuôc tính số
data.describe()

In [ ]:
#Đặc trưng thống kê dữ liệu Object
data.describe(include=['O'])

In [ ]:
data[data.overview=='No overview found.']

# **2) Tiền xử lý dữ liệu tập Movies**
---

## **2.1) Chuẩn hóa dữ liệu:**
---
* Kiểm tra dữ liệu null của các thuộc tính
* Loại bỏ các bộ film có thuộc tín null
* Sắp xếp lại các bộ film theo ngày phát hành

In [ ]:
#Thống kê số liệu missing trong Data frame
#Theo từng cột
print('Số lượng missing data trong file dữ liệu:')
print(data.isnull().sum())

In [ ]:
#Xóa tất cả các bộ film có chứ thuộc tính null
data.dropna(axis=0,how='any',inplace=True)
data.info()

In [ ]:
#Sắp xếp lại dữ liệu theo ngày phát hành
data.sort_values('release_date',axis=0,inplace=True)
data.head()

In [ ]:
data.tail()

## **2.2) Loại bỏ các bộ film trùng tên trong tập dữ liệu**
---
Thực hiện xóa các bộ film trùng tên trong tập dữ liệu chỉ giữ lại bộ film có số lượng vote cao nhất

In [ ]:
#Thống kê các bộ film trùng tên trong tập dữ liệu
data['original_title'].value_counts()

In [ ]:
data.loc[data['original_title']=='Hamlet']

In [ ]:
#Sắp xếp film theo thuộc tính vote_count và xóa các film trùng tên, giữ lại film có lượt vote lớn hơn
data.sort_values('vote_count',ascending=True,inplace=True)
data.drop_duplicates(['original_title'],keep='last',inplace=True)
data.info()

In [ ]:
#check lại dữ liệu sau khi xử lý
data.loc[data['original_title']=='Hamlet']

In [ ]:
#Thống kê các bộ film trùng tên trong tập dữ liệu sau xử lý
data['original_title'].value_counts()

## **2.3) Xử lý các bộ film không có tóm tắt film**
---

In [ ]:
#Thống kê các dữ liệu trùng nhau
data['overview'].value_counts()

In [ ]:
#lọc các bộ film có phần tóm tắt là: No overview found, hoặc No Overview, hoăc chuỗi rỗng, hoặc No movie overview available. 
data.loc[(data['overview']=='No overview found.')].sort_values('overview')

In [ ]:
#Có tất cả 8 bộ film không có dữ liệu tóm tắt film
#Xóa các bộ film này
data = data.loc[(data['overview']!='No overview found.')]
data.info()

In [ ]:
#Check lại dữ liệu sau khi xử lý phần tóm tắt
data['overview'].value_counts()

In [ ]:
data.loc[(data['overview']=='A few funny little novels about different aspects of life.')]

## **2.4) Lưu dữ liệu sau khi đã xử lý ra file**
---

In [ ]:
#Lưu dữ liệu ra file Data_Movies_ok.csv
data.reset_index(drop=True,inplace=True)
data.sort_values(['release_date'],inplace=True)
data.to_csv('data\Data_Movies_ok.csv',index=None)

# **3) Xây dựng các hệ thống Recommender Systems**
---
Recommender systems có thể phân thành 3 loại như sau:

* **Hệ thống đề xuất dựa trên nội dung (Content-based recommenders):** Hệ thống này sẽ gợi ý các bộ phim tương tự với bộ fim mà người dùng xem. Hệ thống này sử dụng metadata của các bộ film như: Thể loại film, đạo diễn, mô tả film, diễn viên...Ý tưởng chính đằng sau hệ thống đề xuất dựa vào nội dung đó là nếu một người đã thích/xem một bộ film nào đó, thì họ cũng sẽ thích/xem một bộ phim tương tự với bộ phim đã xem. 


* **Hệ thống lọc cộng tác (Collaborative filtering engines):** Hệ thống này cố gắng dự đoán thông qua đánh giá hoặc ưa thích mà một người dùng đã đưa ra đối với một bộ film dựa trên đánh giá và ưa thích của những người sử dụng khác. Lọc cộng tác không yêu cầu metadata giống như lọc theo nội dung. (Tìm một người xem có các thuộc tính tương đồng với người dùng này và đề xuất các bộ film theo người xem trước đây)

* **Hệ thống lai (Hybrid Engine):** Kết hợp các ý tưởng của Content-based recommender và Collaborative filtering để xây dựng một hệ thống đề xuất.

![alt text](https://data-flair.training/blogs/wp-content/uploads/sites/2/2019/07/data-science-movie-recommendation-project.jpg)

## **3.1) Simple Recommenders (Giải quyết trường hợp Cold-Start Problem)**
---
Lọc ra  15 bộ film (các bộ film nổi bật) trong danh sách film có trong CSDL có tỷ lệ người đánh giá và điểm đánh giá cao để đề xuất cho người dùng xem.

![alt text](https://user-images.githubusercontent.com/42392773/52229643-67723e80-28db-11e9-8ef8-4ddd945a34cd.jpg)

Các bộ film được tính trọng số đánh giá (Weighted Rating - WR), và dựa vào trọng số này để lọc ra danh sách các bộ film nổi bật đề xuất cho người xem.



Tính Weighted Rating (WR):

**WR = {[v/(v+m) . R] + [m/(v+m) . C]}**

* v: Số lượng người vote cho bộ film đó
* m: Số lượng người vote tối thiểu yêu cầu đối với 1 bộ film 
* R: Vote trung bình của bộ film đó
* C: vote trung bình của tất cả các bộ film trong tập dữ liệu


In [ ]:
#Hiển thị dữ liệu sau khi đã tiền xử lý
data.head()

In [ ]:
# Tính C: Đánh giá trung bình của các bộ film trong tập dữ liệu
C = data['vote_average'].mean()
print(C)

In [ ]:
#Số lượng người vote tối thiểu cho một bộ film phải từ 1000 người trở lên
#Lọc các film có vote_count > m thành một DataFrame mới
m=1000
movies_vote_1000 = data.copy().loc[data['vote_count']>=m]
movies_vote_1000.head()

In [ ]:
print('Data ban đầu:',data.shape)
print('Data film có vote_count>1000:', movies_vote_1000.shape)

In [ ]:
#Hiển thị dữ liệu thống kê cho thuộc tính vote_average, vote_count
movies_vote_1000.describe()

Tập dữ liệu ban đầu có 11 756 bộ film, sau khi lọc chỉ lấy những bộ film có tổng số lượt vote từ 1000 lần trở lên, có tất cả 1109 bộ film thỏa mãn, lưu sang Dataframe mới có tên: movie_vote_1000

In [ ]:
# Xây dựng hàm tính trọng số đánh giá WR
def wr(x,m=m,C=C):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R) + (m/(m+v)*C)

In [ ]:
#Thêm một thuộc tính mới cho mỗi bộ film  'score', lưu giá trị Weighted_rating tương ứng
movies_vote_1000['score'] =  movies_vote_1000.apply(wr,axis=1)
movies_vote_1000.head()

In [ ]:
#Sắp xếp lại dữ liệu theo score giảm dần và lọc ra 15 film có score cao nhất
movies_vote_1000=movies_vote_1000.sort_values('score',ascending=False)
movies_vote_1000

In [ ]:
#Lấy 15 film có điểm trọng số đánh giá cao nhất
list15 = movies_vote_1000[['original_title','vote_count','vote_average','score']].head(15).copy()
list15.reset_index(drop = True, inplace=True)
print('DANH SÁCH 15 BỘ FILM CÓ ĐIỂM ĐÁNH GIÁ (score) CAO NHẤT')
list15

Sau khi tính toán trọng số đánh giá (WR), dựa vào thông số này để đưa ra danh sách các film (có trọng số đánh giá cao) gợi ý cho người xem. Ví dụ trong trường hợp ở trên chúng ta đưa ra 15 bộ phim có trọng số đánh giá cao nhất.

WR phụ thuộc vào vote trung bình của bộ phim đó với lượng người vote cho bộ film. Có những bộ film có vote trung bình cao nhưng lượng người vote lại ít nên WR thấp.

Một số bộ film có điểm vote trung bình cao > 8.5, nhưng số lượt vote lại rất thấp nên ko được đưa vào danh sách tính WR (<1000 lượt vote)

In [ ]:
#Số lượng các film có vote trung bình > 8.5
data[data['vote_average']>8.5].count()

In [ ]:
#Danh sách các film có vote trung bình >8.5
data[data['vote_average']>8.5].loc[:,['original_title','vote_average','vote_count']]

## **3.2) Content-Based Recommender:**
---
Với hệ thống đề xuất dựa trên nội dung, nhiệm vụ của chúng ta là phải tìm được một bộ film có nội dung tương đồng cao nhất với một bộ film xác định. 

Chúng ta sẽ phải tính toán số điểm tương đồng theo từng cặp cho tất cả các bộ film và đưa ra bộ film đề xuất có điểm tương đồng cao nhất.

<img src='pic/pic2.png' width='400px'>


### A) Dựa vào tóm tắt film (Overview)
---
Dữ liệu film có thuộc tính "overview" đây là thuộc tính tóm tắt nội dung của bộ film. Chúng ta sẽ dựa vào thông tin tóm tắt film để tìm bộ film có nội dung tương tự với bộ film đưa vào.

* Sử dụng phương pháp vertor hóa: TF-IDF


In [ ]:
#Dữ liệu các bộ film ban đầu
data.info()

Sử dụng TF-IDF để đánh giá độ tương đồng giữa 2 bộ film dựa vào phần tóm tắt nội dung film.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Định nghĩa một vector TF-IDF loại bỏ tất cả các stop words trong TA
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
#Xây dựng ma trận TF-IDF
tfidf_matrix = tfidf.fit_transform(data['overview'])
# ma trận corpus của TFIDF
tfidf_matrix.shape

In [ ]:
tfidf_matrix

In [ ]:
#Hiển thị dữ liệu ma trân thưa
tfidf_matrix[0:10,130:140].toarray()

In [ ]:
print(tfidf_matrix)

Có tất cả 34 234 từ khác nhau (ko kể các stop words) được sử dụng để tóm tắt nội dung của 12 109 bộ film. Dựa vào tâp corpus này chúng ta sẽ thực hiện việc tính toán độ tương đồng. Có thể sử dụng các độ đo như:
* Euclidean distance. 
* Cosine distance.

Câu hỏi đặt ra là độ đo tương đồng nào là tốt nhất? ko có độ đo nào là tốt nhất nó phụ thuộc vào từng loại dữ liệu và bài toán cụ thể. 

Chúng ta sẽ sử dụng độ đo Cosine để tính độ tương đồng:

![alt text](https://sites.temple.edu/tudsc/files/2017/03/cosine-equation.png)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

#Tính độ tương tự cosine giữa các bộ film với nhau dựa vào tóm tắt film
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
print(cosine_sim.shape)
print(cosine_sim)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.heatmap(cosine_sim[0:100,0:100], linewidth=0.5,cmap='hot' )
#plt.show()

#plt.imshow(cosine_sim[0:100,0:100], cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
#Lấy danh sách tên các bộ film
indices = pd.Series(data.index,index=data['original_title'])
indices

Xây dựng một hàm đưa vào tên của một bộ film sau đó dựa vào ma trận cosine_sim để xác định 15 bộ film có độ tương đồng cao theo thứ tự giảm dần.

* input: title, cosine_sim
* output: list 15 film similarity


In [ ]:
def get_recommend_movies(title,cosine_sim=cosine_sim):
    #Lấy index của bộ film theo tiêu đề đưa vào
    idx=indices[title]
    #Lấy điểm tương đồng theo cặp của tất cả các movies theo tiêu đề bộ film đưa vào
    sim_scores = list(enumerate(cosine_sim[idx]))
    #Sắp xếp các bộ film dựa theo điểm tương đồng
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #Lấy điểm của 15 bộ film có độ tương đồng cao nhất
    sim_scores_15 = sim_scores[1:16]
    #Lấy index tương ứng với 15 bộ film này
    movies_index = [i[0] for i in sim_scores_15]
    #trả ra tiêu đề của 15 bộ film
    return data[['original_title','overview']].iloc[movies_index]

In [ ]:
#Thử đề xuất với tên bộ film bất kỳ
#1. Bộ film: Batman Forever
get_recommend_movies('Batman Forever')

In [ ]:
#Thử đề xuất với tên bộ film bất kỳ
#2. Bộ film: The Shawshank Redemption
get_recommend_movies('The Shawshank Redemption')

In [ ]:
#Thử đề xuất với tên bộ film bất kỳ
#3. Bộ film: Star Wars
get_recommend_movies('Star Wars')